In [13]:
# baseball-regression.ipynb
# Alexis Perumal, Venkat Pinnika, Young You, 1/8/2020
#
# Objective:
# . Build a predictor for the winner of baseball games (visitor or home)
#   with > 50% prediction accuracy, with statistical accuracy.
# . Analysis will be done by season, looking at all the regular season
#   games of a given year with each days' game predicted based on info
#   up to, but not including that day.
#
# Strategy:
# . Build a dataframe of games (rows) and factors to build a regression
#   model for visiting net points, and by extension, predict the winner
#   of each game (visitor or home).
# . Explore candidate factors with regression analysis (T-stat, F-stat,
#   p-value), build and optimize a simple LSR model using a training set
#   through the 2017 Season.
# . Plot the results.
# . Ultimately, apply the model to the 2018, then 2019 seasons.

In [14]:
# Modules
import os
import csv
import pprint
import pandas as pd
import glob
import pprint
import datetime
%matplotlib inline
!pwd

/Users/aperumal/OneDrive/Personal/UCSD_Data_Science_Bootcamp/Homework/2019-12-18_Proj1_Baseball-Predictor/baseball-predictor/alexis-regression


In [15]:
# Support functions
def date_str(date: int):
    s = str(date)
    return f"{s[0:4]}-{s[4:6]}-{s[6:]}"


# Passed the gamedays series (YYYYMMDD) and season (year) return with the first
# and last game dates.
def date_range(gamedays, season):
    # Converts 'YYYY' string to an integer start date, YYYY0101 and end date YYYY1231
    def season_to_date(season): 
#         return (int(season)*10000 + 101, int(season)*10000+1231)
        # Hack to shorten the season for dev purposes (faster analysis)
        return (int(season)*10000 + 101, int(season)*10000+430)
    
    first_of_year, last_of_year = season_to_date(season)
    season_gamedays = gamedays.loc[((gamedays >= first_of_year) &
                                    (gamedays <= last_of_year))]
    return (season_gamedays.iloc[0], season_gamedays.iloc[-1])
    

# returns new game date offset by n. Passed in the gamedays series.
def gamedays_offset(gamedays, base_date, n): 
    if base_date not in gamedays.values:
        raise ValueError(f"{base_date}, not in the the gamedays series.")
    base_date_index = gamedays[gamedays==base_date].index[0]
    if ((n + base_date_index) < 0) or n + base_date_index >= len(gamedays):
        raise ValueError(f"Attempting to calculate a game date outside the dataset.")
        return(0)  # Out of range
    else:
        new_index = base_date_index + n
        return gamedays.iloc[new_index]
    
def derive_metrics(results_df):
    num_games = len(results_df)
    num_correct = results_df['Prediction Correct?'].values.sum()
    percent_correct = num_correct/num_games*100.
    return (num_games, num_correct, percent_correct)

In [16]:
def read_source_data():
    def reader(f):
        df = pd.read_csv(f, index_col=False, header=None)  
        df.columns = [("Col_"+str(i)) for i in range(1,df.shape[1]+1)]       
        return df

    files = glob.glob("../datasets/Final_Data_Files/GL*.csv")
    files.sort()
    df = pd.concat([reader(f) for f in files])
    old_df_len = len(df)
    
    # Insert column headers
    df = df.rename(columns={'Col_1':'Date',
                            'Col_4':'Visiting Team',
                            'Col_5':'Visiting League',
                            'Col_7':'Home Team',
                            'Col_8':'Home League',
                            'Col_10':'Visiting Score',
                            'Col_11':'Home Score',
                            'Col_102':'V Start Pitcher ID',
                            'Col_104':'H Start Pitcher ID'
                           }).reset_index()
    df = df[['Date', 'Visiting Team', 'Visiting League', 'Home Team', 'Home League',
            'Visiting Score','Home Score', 'V Start Pitcher ID','H Start Pitcher ID']]
    
#     print(df.head(30))
    df = df.replace('FLO','MIA') # After the 2011 season, the Florida Marlins
                                 # rebranded themselves the Miami Marlins. This
                                 # search and replace makes the two the same.
#     print(df.head(30))
    
    # Drop all rows with missing information
    # print(df.head())
    df = df.dropna(how='any')
    if len(df) < old_df_len:
        print(f"Dropped {old_df_len-len(df)} rows due to missing data.")
    
    # Create new columns we'll need. 
#     df['Key'] = df['Date'].astype(str) + df['Visiting Team']
    df['Home Winner'] = df['Home Score'] > df['Visiting Score']
    df['V NetRuns'] = df['Visiting Score'] - df['Home Score']
    df['H NetRuns'] = - df['V NetRuns']
    
    
    #     print(df.shape)
    print(f"Dataset loaded with {df.shape[0]} games, ", end='')
    print(f"{df.shape[1]} columns, {date_str(df.iloc[0, 0])} - ", end='')
    print(f"{date_str(df.iloc[-1,0])}")
    return df

# read_source_data()

In [17]:
# Create a DF to hold team stats starting at the beginning of the lookback window.
def build_netpoints_dfs(games_df, start_date, end_date, lookback_n, gdays, excl_day_before=False):
    start_processing = datetime.datetime.now()
    print(f"  {start_processing}: Starting build of net runs rolling average tables.")

    # Determine lookback dates for first game day.
    lookback_start_day = gamedays_offset(gdays, start_date, -lookback_n)
    lookback_end_day = gamedays_offset(gdays, start_date, -1)
    
    # Update gamedays to the dates we care about (prediction range and lookback)
    gdays = gdays.loc[((gdays >= lookback_start_day) & (gdays <= end_date))]

    # Create home and visitor tables to hold net points, by team and gameday.
    # Start with the visiting teams.
    v_teams = games_df['Visiting Team'].unique()
    # print(v_teams)
    v_teams.sort()
    v_np_df = pd.DataFrame(columns=v_teams, index=gdays) # Visiting net points by day
    v_ng_df = pd.DataFrame(columns=v_teams, index=gdays) # Visiting # games by day

    # Calculate the home team net points.
    h_teams = games_df['Home Team'].unique()
    h_teams.sort()
    h_np_df = pd.DataFrame(columns=h_teams, index=gdays) # Home net points by day
    h_ng_df = pd.DataFrame(columns=h_teams, index=gdays) # Home # games by day

    # Now populate the visiting and home team net points by game day dataframe
    for day in v_np_df.index:
        for team in v_np_df.columns: # Visiting Team Data
            net_runs = games_df.loc[((games_df['Visiting Team']==team)&
                       (games_df['Date']==day)),:]['V NetRuns'].sum()
            num_games = games_df.loc[((games_df['Visiting Team']==team)&
                        (games_df['Date']==day)),:]['V NetRuns'].count()
            v_np_df.at[day, team] = net_runs
            v_ng_df.at[day, team] = num_games

        for team in h_np_df.columns: # Home Team Data
            net_runs = games_df.loc[((games_df['Home Team']==team)&
                                     (games_df['Date']==day)),:]['H NetRuns'].sum()
            num_games = games_df.loc[((games_df['Home Team']==team)&
                               (games_df['Date']==day)),:]['H NetRuns'].count()
            h_np_df.at[day, team] = net_runs
            h_ng_df.at[day, team] = num_games

    # Now populate the visiting team net points rolling averages dataframe
    # Avg = sum of net points divided by # of games
    if excl_day_before:
#         print(f"Excluding day before. n={lookback_n}")
#         print("")
#         print("Net Points")
#         print(v_np_df['LAN'].head())
#         print("Net Points - rolling 3")
#         print(v_np_df['LAN'].rolling(3).sum().head())
#         print("Net Points - shifted")
#         print(v_np_df['LAN'].shift(periods=1).head())
#         print("Net Points - shifted - rolling 3")
#         print(v_np_df['LAN'].shift(periods=1).rolling(2).sum().head())
        
#         print("")
#         print("Num Games")
#         print(v_ng_df['LAN'].head())
#         print("Num Games - rolling 3")
#         print(v_ng_df['LAN'].rolling(3).sum().head())
#         print(v_ng_df['LAN'].shift(periods=1).head())
#         print(v_ng_df['LAN'].shift(periods=1).rolling(2).sum().head())
        
#         print("")
#         print("rolling average net points (standard)")
#         v_ra_np_df = v_np_df.rolling(3).sum() / \
#             v_ng_df.rolling(3).sum()  # Rolling average of mean net points
#         print(v_ra_np_df['LAN'].head())

        
#         print("")
#         print("rolling average net points excluding day before")
#         v_ra_np_df2 = v_np_df.shift(periods=1).rolling(2).sum() / \
#             v_ng_df.shift(periods=1).rolling(2).sum()  # Rolling average of mean net points
#         print(v_ra_np_df2['ANA'].head())
        
#         sys.exit() # Stop here to view the data and debug it.
        
        v_ra_np_df = v_np_df.shift(periods=1).rolling(lookback_n-1).sum() / \
            v_ng_df.shift(periods=1).rolling(lookback_n-1).sum()  # Rolling average of mean net points

        h_ra_np_df = h_np_df.shift(periods=1).rolling(lookback_n-1).sum() / \
            h_ng_df.shift(periods=1).rolling(lookback_n-1).sum()  # Rolling average of mean net points
    else:
        v_ra_np_df = v_np_df.rolling(lookback_n).sum() / \
            v_ng_df.rolling(lookback_n).sum()  # Rolling average of mean net points
        h_ra_np_df = h_np_df.rolling(lookback_n).sum() / \
            h_ng_df.rolling(lookback_n).sum()  # Rolling average of mean net points

    end_processing = datetime.datetime.now()
    duration = end_processing - start_processing
    print(f'  {end_processing}: Net point tables calculated in {duration} hr/min/sec.')

    return (v_ra_np_df, h_ra_np_df)

In [18]:
# # Predictor to be called by the harness, passed in the lookback_n and season (year).
# # lookback_n indicates how many prior game days to consider when calculating net
# # points (runs) for prediction purposes.
# def net_points_predictor(lookback_n, season, excl_day_before=False):
#     games_df = read_source_data()
#     gamedays = pd.Series(games_df['Date'].unique())  # Series with gamedays.
#     start_date, end_date = date_range(gamedays, season)
    
#     print(f"  Analyzing {season} season: {date_str(start_date)}", end='')
#     print(f" - {date_str(end_date)}, with {lookback_n} day lookback.")
# #     print("")
# #     print(f"Starting games_df length: {len(games_df)}")
#     (v_ra_np_df, h_ra_np_df) = build_netpoints_dfs(games_df, start_date, end_date, \
#                                                   lookback_n, gamedays, excl_day_before)
# #     print(f"Ending games_df length: {len(games_df)}")
    
#     results_df = calc_predictions(games_df, v_ra_np_df, h_ra_np_df, start_date, \
#                                   end_date, lookback_n, gamedays)
    
#     print(results_df.tail(10))
    
#     return derive_metrics(results_df)

In [19]:
def add_net_runs_avg_old(g, d): # g is games_df, d is the dates dictionary
    
    for i in range(d['start_date_index'], d['end_date_index']+1):
        
        day = d['gamedays'].iloc[i]
        lookback_start_date = d['gamedays'].iloc[i-d['lookback_n']]
        lookback_end_date = d['gamedays'].iloc[i-1]  # Lookback ends the day before gameday
        print(f"Day: {day}, Lookback Start: {lookback_start_date}, Lookback End: {lookback_end_date}")

        day_df = g.loc[(g['Date']==day),:] # Small dataframe with all the games on gameday
        pprint.pprint(day_df)
        
        # Small dataframe looking at the lookback window.
        lookback_df = g.loc[((g['Date']>= lookback_start_date) &
                             (g['Date']<= lookback_end_date)), :]
        pprint.pprint(lookback_df)

        for g_i in range(len(day_df)): # iterate across all games on a given gameday
            game = day_df.iloc[g_i]  # Looking at a particular game on gameday
            print(game)
            print(type(game))
            print(game.index)
            v_team = game['Visiting Team']
            h_team = game['Home Team']
            
            # Calculate net point average for v_team
            v_team_npa = lookback_df.loc[(lookback_df['Visiting Team']==v_team),'V NetRuns'].mean()
            h_team_npa = lookback_df.loc[(lookback_df['Home Team']==h_team),'H NetRuns'].mean()
            print(f"V net runs avg = {v_team_npa}, H net runs avg = {h_team_npa}")
            
    return g

In [20]:
# # Calculate the Stochastic Oscillator
# n = 14

# # Make a copy of the high and low column.
# low_14, high_14 = price_data[['symbol','low']].copy(), price_data[['symbol','high']].copy()

# # Group by symbol, then apply the rolling function and grab the Min and Max.
# low_14 = low_14.groupby('symbol')['low'].transform(lambda x: x.rolling(window = n).min())
# high_14 = high_14.groupby('symbol')['high'].transform(lambda x: x.rolling(window = n).max())

# # Calculate the Stochastic Oscillator.
# k_percent = 100 * ((price_data['close'] - low_14) / (high_14 - low_14))

# # Add the info to the data frame.
# price_data['low_14'] = low_14
# price_data['high_14'] = high_14
# price_data['k_percent'] = k_percent

# # Display the head.
# price_data.head(30)



In [21]:
def add_net_runs_avg(g, d): # g is games_df, d is the dates dictionary

    # g_v_team = g.groupby(['Visiting Team'])
    # print(g.groupby('Visiting Team')['Date','Visiting Team','V NetRuns'].sum().head())
    
    g['V NP Avg'] = g.groupby('Visiting Team')['V NetRuns'].\
       transform(lambda x: x.rolling(window = 10).mean())
    g['H NP Avg'] = g.groupby('Home Team')['H NetRuns'].\
       transform(lambda x: x.rolling(window = d['lookback_n']).mean())
                                                                      
    return g

In [22]:
# Top level (main) code, read in the dataset, add the columns we care about.

# Constants
max_lookback_n = 10 # Maximum lookback to consider
test_n = 10 # Artifically constrained # of days to calculate (for test & debug)
DEV_AND_TEST = True

games_df = read_source_data()

# Capture key date parameters.
# dates = {}
# dates['gamedays'] = pd.Series(games_df['Date'].unique())  # Series with gamedays.
# # dates['lookback_start_date'] = gamedays[0]
# dates['start_date_index'] = max_lookback_n
# dates['start_date'] = gamedays[dates['start_date_index']]
# dates['lookback_n'] = max_lookback_n
# if DEV_AND_TEST:
#     dates['end_date_index'] = max_lookback_n + test_n - 1
#     dates['end_date'] = gamedays[dates['end_date_index']]
# else:
#     dates['end_date_index'] = len(gamedays)-1
#     dates['end_date'] = gamedays[len(gamedays)-1]
# pprint.pprint(dates)


# games_df
# # games_df = add_net_runs_avg(games_df, dates)
# games_df['V NP Avg'] = games_df.groupby('Visiting Team')['V NetRuns'].transform(lambda x: x.rolling(window = 10).mean())
# # print(games_df.head(15))
# games_df.sort_values(by = ['Visiting Team','Date']).head(20)
# games_df = games_df.reset_index()
#games_df.sort_index()
# games_df.sort_values(by='index')
games_df

Dataset loaded with 19437 games, 12 columns, 2010-04-04 - 2017-10-01


,Date,Visiting Team,Visiting League,Home Team,Home League,Visiting Score,Home Score,V Start Pitcher ID,H Start Pitcher ID,Home Winner,V NetRuns,H NetRuns
0,20100404,NYA,AL,BOS,AL,7,9,sabac001,beckj002,True,-2,2
1,20100405,MIN,AL,ANA,AL,3,6,bakes002,weavj003,True,-3,3
2,20100405,CLE,AL,CHA,AL,0,6,westj001,buehm001,True,-6,6
3,20100405,DET,AL,KCA,AL,8,4,verlj001,greiz001,False,4,-4
4,20100405,SEA,AL,OAK,AL,5,3,hernf002,sheeb001,False,2,-2
...,...,...,...,...,...,...,...,...,...,...,...,...
19432,20171001,ARI,NL,KCA,AL,14,2,ray-r002,vargj001,False,12,-12
19433,20171001,DET,AL,MIN,AL,1,5,sanca004,colob001,True,-4,4
19434,20171001,TOR,AL,NYA,AL,2,1,andeb004,montj004,False,1,-1
19435,20171001,BAL,AL,TBA,AL,0,6,gausk001,snelb001,True,-6,6


In [26]:
games_df2 = games_df.sort_values(by = ['Visiting Team','Date'])
games_df2['V NP Avg'] = games_df2.groupby('Visiting Team')['V NetRuns'].transform(lambda x: x.rolling(window = 10).mean())

games_df2 = games_df2.sort_values(by = ['Home Team','Date'])
games_df2['H NP Avg'] = games_df2.groupby('Home Team')['H NetRuns'].transform(lambda x: x.rolling(window = 10).mean())

games_df2.loc[games_df2['Visiting Team']=="LAN", :].head(20)

games_df2 = games_df2.sort_index()

# games_df2 = games_df2.sort_values(by = 'Date')


In [28]:
# games_df2.loc[games_df2['Home Team']=="LAN", :].head(20)

games_df2.head(40)

,Date,Visiting Team,Visiting League,Home Team,Home League,Visiting Score,Home Score,V Start Pitcher ID,H Start Pitcher ID,Home Winner,V NetRuns,H NetRuns,V NP Avg,H NP Avg
0,20100404,NYA,AL,BOS,AL,7,9,sabac001,beckj002,True,-2,2,NaN,NaN
1,20100405,MIN,AL,ANA,AL,3,6,bakes002,weavj003,True,-3,3,NaN,NaN
2,20100405,CLE,AL,CHA,AL,0,6,westj001,buehm001,True,-6,6,NaN,NaN
3,20100405,DET,AL,KCA,AL,8,4,verlj001,greiz001,False,4,-4,NaN,NaN
4,20100405,SEA,AL,OAK,AL,5,3,hernf002,sheeb001,False,2,-2,NaN,NaN
5,20100405,TOR,AL,TEX,AL,4,5,marcs001,felds001,True,-1,1,NaN,NaN
6,20100405,SDN,NL,ARI,NL,3,6,garlj001,hared001,True,-3,3,NaN,NaN
7,20100405,CHN,NL,ATL,NL,5,16,zambc001,lowed001,True,-11,11,NaN,NaN
8,20100405,SLN,NL,CIN,NL,11,6,carpc002,haraa001,False,5,-5,NaN,NaN
9,20100405,SFN,NL,HOU,NL,5,2,linct001,oswar001,False,3,-3,NaN,NaN


In [ ]:
# From Venkat, on pitchers
# v_pitchers = game_day_df2['V Start Pitcher ID'].to_list()
# v_pitcher_net_score = []
# for pitcher in v_pitchers:
#     v_pitcher_net_score.append(train_df.loc[(train_df['V Start Pitcher ID']==pitcher),:]['Home Score'].mean())
# game_day_df2['V Avg Pitcher Runs given'] = v_pitcher_net_score
# h_pitchers = game_day_df2['H Start Pitcher ID'].to_list()
# h_pitcher_net_score = []
# for pitcher in h_pitchers:
#     h_pitcher_net_score.append(train_df.loc[(train_df['H Start Pitcher ID']==pitcher),:]['Visiting Score'].mean())
# game_day_df2['H Avg Pitcher Runs given'] = h_pitcher_net_score
# game_day_df2['Predict Home Pitcher Wins?'] = game_day_df2['H Avg Pitcher Runs given'] < game_day_df2['V Avg Pitcher Runs given']
# game_day_df2